In [1]:
import soundfile as sf
import os, librosa
import numpy as np
import torch
import torch.nn as nn
import random
from transformers import Wav2Vec2ForCTC, Wav2Vec2Processor, BertTokenizer, BertModel
from sklearn.metrics import classification_report, accuracy_score, confusion_matrix
import timeit
# from pynvml import *

In [2]:
DEVICE = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
processor = Wav2Vec2Processor.from_pretrained("facebook/wav2vec2-base-960h")
asr_model = Wav2Vec2ForCTC.from_pretrained("facebook/wav2vec2-base-960h").to(DEVICE)

Some weights of Wav2Vec2ForCTC were not initialized from the model checkpoint at facebook/wav2vec2-base-960h and are newly initialized: ['wav2vec2.masked_spec_embed']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [94]:
# batch data preparation
path = "/afs/inf.ed.ac.uk/user/s20/s2057508/Documents/Corpora/IEMOCAP/Data"
os.chdir(path)
names_train = []
names_test = []
emolabels_train = []
emolabels_test = []
# utters_train = []
# utters_test = []
# feats_train = []
# feats_test = []
train_asr_hidden0 = []
train_asr_hidden1 = []
train_asr_hidden2 = []
train_asr_hidden3 = []
train_asr_hidden4 = []
train_asr_hidden5 = []
train_asr_hidden6 = []
train_asr_hidden7 = []
train_asr_hidden8 = []
train_asr_hidden9 = []
train_asr_hidden10 = []
train_asr_hidden11 = []
train_asr_hidden12 = []

test_asr_hidden0 = []
test_asr_hidden1 = []
test_asr_hidden2 = []
test_asr_hidden3 = []
test_asr_hidden4 = []
test_asr_hidden5 = []
test_asr_hidden6 = []
test_asr_hidden7 = []
test_asr_hidden8 = []
test_asr_hidden9 = []
test_asr_hidden10 = []
test_asr_hidden11 = []
test_asr_hidden12 = []

with open('emo_name_trans_5500.txt', 'r') as f:
    files = f.readlines()
    files = np.array(files)
    indices = np.arange(len(files))
    random.shuffle(indices)
    files[np.arange(len(files))] = files[indices]
    for line in files:
        emotion = line.split(" ")[0]
        file_name = line.split(" ")[1]
#         list_string = line.split(" ")[2:]
#         utterance = " ".join(list_string)[:-1].upper()
        
        if emotion == "Ang":
            emotion_label = 0
        elif emotion == "Hap":
            emotion_label = 1
        elif emotion == "Neu":
            emotion_label = 2
        elif emotion == "Sad":
            emotion_label = 3
        
        audio_input, sr = sf.read(file_name)
        inputs = processor(audio_input, sampling_rate=16000, return_tensors='pt').input_values.to(DEVICE)
        hiddens = asr_model(inputs, output_hidden_states=True).hidden_states
        hidden0 = hiddens[0].detach()[0].mean(dim=0)
        hidden1 = hiddens[1].detach()[0].mean(dim=0)
        hidden2 = hiddens[2].detach()[0].mean(dim=0)
        hidden3 = hiddens[3].detach()[0].mean(dim=0)
        hidden4 = hiddens[4].detach()[0].mean(dim=0)
        hidden5 = hiddens[5].detach()[0].mean(dim=0)
        hidden6 = hiddens[6].detach()[0].mean(dim=0)
        hidden7 = hiddens[7].detach()[0].mean(dim=0)
        hidden8 = hiddens[8].detach()[0].mean(dim=0)
        hidden9 = hiddens[9].detach()[0].mean(dim=0)
        hidden10 = hiddens[10].detach()[0].mean(dim=0)
        hidden11 = hiddens[11].detach()[0].mean(dim=0)
        hidden12 = hiddens[12].detach()[0].mean(dim=0)

        if "Ses03" not in file_name:
            names_train.append(file_name)
            emolabels_train.append(emotion_label)
            train_asr_hidden0.append(hidden0.view(1, 768).cpu().numpy())
            train_asr_hidden1.append(hidden1.view(1, 768).cpu().numpy())
            train_asr_hidden2.append(hidden2.view(1, 768).cpu().numpy())
            train_asr_hidden3.append(hidden3.view(1, 768).cpu().numpy())
            train_asr_hidden4.append(hidden4.view(1, 768).cpu().numpy())
            train_asr_hidden5.append(hidden5.view(1, 768).cpu().numpy())
            train_asr_hidden6.append(hidden6.view(1, 768).cpu().numpy())
            train_asr_hidden7.append(hidden7.view(1, 768).cpu().numpy())
            train_asr_hidden8.append(hidden8.view(1, 768).cpu().numpy())
            train_asr_hidden9.append(hidden9.view(1, 768).cpu().numpy())
            train_asr_hidden10.append(hidden10.view(1, 768).cpu().numpy())
            train_asr_hidden11.append(hidden11.view(1, 768).cpu().numpy())
            train_asr_hidden12.append(hidden12.view(1, 768).cpu().numpy())
        elif "Ses03" in file_name:
            names_test.append(file_name)
            emolabels_test.append(emotion_label)
            test_asr_hidden0.append(hidden0.view(1, 768).cpu().numpy())
            test_asr_hidden1.append(hidden1.view(1, 768).cpu().numpy())
            test_asr_hidden2.append(hidden2.view(1, 768).cpu().numpy())
            test_asr_hidden3.append(hidden3.view(1, 768).cpu().numpy())
            test_asr_hidden4.append(hidden4.view(1, 768).cpu().numpy())
            test_asr_hidden5.append(hidden5.view(1, 768).cpu().numpy())
            test_asr_hidden6.append(hidden6.view(1, 768).cpu().numpy())
            test_asr_hidden7.append(hidden7.view(1, 768).cpu().numpy())
            test_asr_hidden8.append(hidden8.view(1, 768).cpu().numpy())
            test_asr_hidden9.append(hidden9.view(1, 768).cpu().numpy())
            test_asr_hidden10.append(hidden10.view(1, 768).cpu().numpy())
            test_asr_hidden11.append(hidden11.view(1, 768).cpu().numpy())
            test_asr_hidden12.append(hidden12.view(1, 768).cpu().numpy())

f.close()

emolabels_train = torch.tensor(emolabels_train)
emolabels_test = torch.tensor(emolabels_test)
train_asr_hidden0 = torch.tensor(train_asr_hidden0).to(DEVICE)
train_asr_hidden1 = torch.tensor(train_asr_hidden1).to(DEVICE)
train_asr_hidden2 = torch.tensor(train_asr_hidden2).to(DEVICE)
train_asr_hidden3 = torch.tensor(train_asr_hidden3).to(DEVICE)
train_asr_hidden4 = torch.tensor(train_asr_hidden4).to(DEVICE)
train_asr_hidden5 = torch.tensor(train_asr_hidden5).to(DEVICE)
train_asr_hidden6 = torch.tensor(train_asr_hidden6).to(DEVICE)
train_asr_hidden7 = torch.tensor(train_asr_hidden7).to(DEVICE)
train_asr_hidden8 = torch.tensor(train_asr_hidden8).to(DEVICE)
train_asr_hidden9 = torch.tensor(train_asr_hidden9).to(DEVICE)
train_asr_hidden10 = torch.tensor(train_asr_hidden10).to(DEVICE)
train_asr_hidden11 = torch.tensor(train_asr_hidden11).to(DEVICE)
train_asr_hidden12 = torch.tensor(train_asr_hidden12).to(DEVICE)
test_asr_hidden0 = torch.tensor(test_asr_hidden0).to(DEVICE)
test_asr_hidden1 = torch.tensor(test_asr_hidden1).to(DEVICE)
test_asr_hidden2 = torch.tensor(test_asr_hidden2).to(DEVICE)
test_asr_hidden3 = torch.tensor(test_asr_hidden3).to(DEVICE)
test_asr_hidden4 = torch.tensor(test_asr_hidden4).to(DEVICE)
test_asr_hidden5 = torch.tensor(test_asr_hidden5).to(DEVICE)
test_asr_hidden6 = torch.tensor(test_asr_hidden6).to(DEVICE)
test_asr_hidden7 = torch.tensor(test_asr_hidden7).to(DEVICE)
test_asr_hidden8 = torch.tensor(test_asr_hidden8).to(DEVICE)
test_asr_hidden9 = torch.tensor(test_asr_hidden9).to(DEVICE)
test_asr_hidden10 = torch.tensor(test_asr_hidden10).to(DEVICE)
test_asr_hidden11 = torch.tensor(test_asr_hidden11).to(DEVICE)
test_asr_hidden12 = torch.tensor(test_asr_hidden12).to(DEVICE)

In [4]:
# 不需要
mean_train = feats_train.mean(dim=1, keepdim=True)
std_train = feats_train.std(dim=1, keepdim=True)
feats_train = (feats_train - mean_train) / std_train

mean_test = feats_test.mean(dim=1, keepdim=True)
std_test = feats_test.std(dim=1, keepdim=True)
feats_test = (feats_test - mean_test) / std_test

In [5]:
indices_train = []
for i in range(len(names_train)):
    indices_train.append(i)

In [6]:
# 不需要
feats_train = feats_train.to(torch.float)
feats_test = feats_test.to(torch.float)

In [68]:
class NeuralNet(nn.Module):
    def __init__(self):
        super(NeuralNet,self).__init__()
#         self.pool = nn.MaxPool1d(2)
        self.lstm = nn.LSTM(input_size=768,
                            hidden_size=32,
                            num_layers=2,
                            batch_first=True,
                            dropout=0.5,
                            bidirectional=True)
        self.lstm2 = nn.LSTM(input_size=64,
                            hidden_size=32,
                            num_layers=2,
                            batch_first=True,
                            dropout=0.5,
                            bidirectional=True)
        self.attn = nn.MultiheadAttention(64, 16, batch_first=True)
        self.dense = nn.Linear(64, 16)
        self.acti = nn.ReLU()
        self.out= nn.Linear(16, 4)

    def forward(self, x):
#         x = self.pool(x)
        self.lstm.flatten_parameters()
        x, _ = self.lstm(x)
#         x, _ = self.lstm2(x)
        x, _ = self.attn(x,x,x)
        x = x.mean(dim=1)
        x = self.dense(x)
        
        res = self.acti(x)
        emotion = self.out(res)
        return emotion

In [112]:
ser_model = NeuralNet().to(DEVICE)
# ser_model = ser_model.to(torch.double)
asr_model = asr_model.to(torch.float)
params = ser_model.parameters()
optimizer = torch.optim.Adam(params, lr=1e-4, weight_decay=1e-5)
# optimizer = torch.optim.Adam(params, lr=1e-4, betas=(0.9, 0.999), eps=1e-08, weight_decay=1e-05, amsgrad=False)

func = nn.CrossEntropyLoss()

epochs = 100
epoch = 0
batch_size = 64

# training
while epoch < epochs:
    torch.cuda.empty_cache()
    start = timeit.default_timer()
    print("-----epoch: ", epoch, "-----")
    loss_list_train = []
    loss_list_test = []
    predictions_train = []
    predictions_test = []

    print('--training begins--')
    ser_model.train()
    j = 0
    while j < len(names_train)/batch_size:
        if (j+1)*batch_size > len(names_train):
#             input_values = torch.tensor(feats_train[j*batch_size: ]).to(DEVICE)
            emolabels = torch.tensor(emolabels_train[j*batch_size: ]).to(DEVICE)
            indices = indices_train[j*batch_size: ]
            asr_feats = train_asr_hidden12[j*batch_size: ].to(DEVICE)
        else:
#             input_values = torch.tensor(feats_train[j*batch_size:(j+1)*batch_size]).to(DEVICE)
            emolabels = torch.tensor(emolabels_train[j*batch_size:(j+1)*batch_size]).to(DEVICE)
            indices = indices_train[j*batch_size:(j+1)*batch_size]
            asr_feats = train_asr_hidden12[j*batch_size:(j+1)*batch_size].to(DEVICE)
            
        
# asr features
#         train_asr_hidden = []
#         for m in indices:
#             audio_input, sr = sf.read(names_train[m])
#             inputs = processor(audio_input, sampling_rate=16000, return_tensors='pt').input_values.to(DEVICE)
#             hiddens = asr_model(inputs, output_hidden_states=True).hidden_states
#             hidden = hiddens[12].detach()[0].mean(dim=0)
#             train_asr_hidden.append(hidden.view(1, 768).cpu().numpy())
#         train_asr_hidden = torch.tensor(train_asr_hidden).to(DEVICE)

# ser loss
        pred = ser_model(asr_feats)
        train_ser_loss = func(pred, torch.tensor(emolabels))
        for i in pred:
            predictions_train.append(i.detach().cpu().numpy())
        j += 1

# backprop
        optimizer.zero_grad()
        torch.nn.utils.clip_grad_norm_(params, 1.0)
#         print(train_loss.item(), train_asr_loss.item(), train_ser_loss.item())
        train_ser_loss.backward()
        optimizer.step()
        torch.cuda.empty_cache()
    print('--training ends--')


# testing
# extract features
    print('--testing begins--')
    ser_model.eval()
#     k = 0
#     while k < len(names_test):
#         emo = []
#         input_values = torch.tensor(feats_test[k]).to(DEVICE)
#         audio_input, sr = sf.read(names_test[k])
#         inputs = processor(audio_input, sampling_rate=16000, return_tensors='pt').input_values.to(DEVICE)
#         hiddens = asr_model(inputs, output_hidden_states=True).hidden_states
#         hidden = hiddens[12].detach()[0].mean(dim=0)
#         test_asr_hidden = hidden.view(1, 1, 768).to(DEVICE)

# ser loss
    pred = ser_model(test_asr_hidden12)
#     emo.append(emolabels_test[k])
    test_ser_loss = func(pred, torch.tensor(emolabels_test).to(DEVICE))
    for i in pred:
        predictions_test.append(i.detach().cpu().numpy())
    torch.cuda.empty_cache()
#     k += 1
        
# compute performance for each epoch
    predictions_train = np.array(predictions_train)
    predictions_test = np.array(predictions_test)
    emolabels_train = np.array(emolabels_train)
    emolabels_test = np.array(emolabels_test)
    predictions_train = [np.argmax(p) for p in predictions_train]
    acc_train = accuracy_score(emolabels_train, predictions_train)
    predictions_test = [np.argmax(p) for p in predictions_test]
    acc_test = accuracy_score(emolabels_test, predictions_test)
 
    print('Epoch:', epoch, '|train_ser_loss: %.4f' %train_ser_loss, '|train_accuracy: %.4f' %acc_train, '|test_ser_loss: %.4f' %test_ser_loss, '|test_accuracy: %.4f' %acc_test)
    print('test: \n', confusion_matrix(emolabels_test, predictions_test))
    print(classification_report(emolabels_test, predictions_test))

    epoch += 1
    print('---testing ends---')
    
    stop = timeit.default_timer()
    print('Time: ', stop - start) 

-----epoch:  0 -----
--training begins--


/usr/lib/python3.6/site-packages/ipykernel_launcher.py:52: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).


--training ends--
--testing begins--
Epoch: 0 |train_ser_loss: 1.3978 |train_accuracy: 0.3055 |test_ser_loss: 1.4051 |test_accuracy: 0.2487
test: 
 [[  0 240   0   0]
 [  0 286   0   0]
 [  0 319   0   0]
 [  0 305   0   0]]
             precision    recall  f1-score   support

          0       0.00      0.00      0.00       240
          1       0.25      1.00      0.40       286
          2       0.00      0.00      0.00       319
          3       0.00      0.00      0.00       305

avg / total       0.06      0.25      0.10      1150

---testing ends---
Time:  0.7403680058196187
-----epoch:  1 -----
--training begins--
--training ends--
--testing begins--
Epoch: 1 |train_ser_loss: 1.3751 |train_accuracy: 0.3177 |test_ser_loss: 1.3988 |test_accuracy: 0.2774
test: 
 [[  0   0 240   0]
 [  0   0 286   0]
 [  0   0 319   0]
 [  0   0 305   0]]
             precision    recall  f1-score   support

          0       0.00      0.00      0.00       240
          1       0.00      0.00    

--training ends--
--testing begins--
Epoch: 13 |train_ser_loss: 1.2584 |train_accuracy: 0.3828 |test_ser_loss: 1.3194 |test_accuracy: 0.3339
test: 
 [[  2 158  48  32]
 [  1 146  80  59]
 [  1 137  73 108]
 [  1  45  96 163]]
             precision    recall  f1-score   support

          0       0.40      0.01      0.02       240
          1       0.30      0.51      0.38       286
          2       0.25      0.23      0.24       319
          3       0.45      0.53      0.49       305

avg / total       0.35      0.33      0.29      1150

---testing ends---
Time:  0.6723705213516951
-----epoch:  14 -----
--training begins--
--training ends--
--testing begins--
Epoch: 14 |train_ser_loss: 1.2466 |train_accuracy: 0.3851 |test_ser_loss: 1.3176 |test_accuracy: 0.3339
test: 
 [[  4 158  46  32]
 [  1 146  82  57]
 [  6 136  72 105]
 [  2  45  96 162]]
             precision    recall  f1-score   support

          0       0.31      0.02      0.03       240
          1       0.30      0.51 

--training ends--
--testing begins--
Epoch: 26 |train_ser_loss: 1.1868 |train_accuracy: 0.4237 |test_ser_loss: 1.2993 |test_accuracy: 0.3609
test: 
 [[ 67 119  31  23]
 [ 26 143  66  51]
 [ 36 133  59  91]
 [ 28  41  90 146]]
             precision    recall  f1-score   support

          0       0.43      0.28      0.34       240
          1       0.33      0.50      0.40       286
          2       0.24      0.18      0.21       319
          3       0.47      0.48      0.47       305

avg / total       0.36      0.36      0.35      1150

---testing ends---
Time:  0.7088069012388587
-----epoch:  27 -----
--training begins--
--training ends--
--testing begins--
Epoch: 27 |train_ser_loss: 1.1786 |train_accuracy: 0.4211 |test_ser_loss: 1.2929 |test_accuracy: 0.3696
test: 
 [[ 74 112  29  25]
 [ 27 142  64  53]
 [ 40 131  58  90]
 [ 29  39  86 151]]
             precision    recall  f1-score   support

          0       0.44      0.31      0.36       240
          1       0.33      0.50 

--training ends--
--testing begins--
Epoch: 39 |train_ser_loss: 1.1709 |train_accuracy: 0.4425 |test_ser_loss: 1.2820 |test_accuracy: 0.3861
test: 
 [[ 83  97  33  27]
 [ 20 132  78  56]
 [ 29 119  76  95]
 [ 26  25 101 153]]
             precision    recall  f1-score   support

          0       0.53      0.35      0.42       240
          1       0.35      0.46      0.40       286
          2       0.26      0.24      0.25       319
          3       0.46      0.50      0.48       305

avg / total       0.39      0.39      0.38      1150

---testing ends---
Time:  0.7620744919404387
-----epoch:  40 -----
--training begins--
--training ends--
--testing begins--
Epoch: 40 |train_ser_loss: 1.1267 |train_accuracy: 0.4508 |test_ser_loss: 1.2834 |test_accuracy: 0.3843
test: 
 [[ 83  96  34  27]
 [ 20 130  80  56]
 [ 29 121  78  91]
 [ 23  24 107 151]]
             precision    recall  f1-score   support

          0       0.54      0.35      0.42       240
          1       0.35      0.45 

--training ends--
--testing begins--
Epoch: 52 |train_ser_loss: 1.1218 |train_accuracy: 0.4584 |test_ser_loss: 1.2695 |test_accuracy: 0.3983
test: 
 [[ 91  84  37  28]
 [ 24 121  83  58]
 [ 30 118  81  90]
 [ 21  23  96 165]]
             precision    recall  f1-score   support

          0       0.55      0.38      0.45       240
          1       0.35      0.42      0.38       286
          2       0.27      0.25      0.26       319
          3       0.48      0.54      0.51       305

avg / total       0.41      0.40      0.40      1150

---testing ends---
Time:  0.7122618816792965
-----epoch:  53 -----
--training begins--
--training ends--
--testing begins--
Epoch: 53 |train_ser_loss: 1.1374 |train_accuracy: 0.4561 |test_ser_loss: 1.2673 |test_accuracy: 0.3974
test: 
 [[ 94  82  36  28]
 [ 26 120  80  60]
 [ 31 122  74  92]
 [ 21  24  91 169]]
             precision    recall  f1-score   support

          0       0.55      0.39      0.46       240
          1       0.34      0.42 

--training ends--
--testing begins--
Epoch: 65 |train_ser_loss: 1.1413 |train_accuracy: 0.4703 |test_ser_loss: 1.2619 |test_accuracy: 0.3991
test: 
 [[ 92  83  38  27]
 [ 26 120  82  58]
 [ 33 117  85  84]
 [ 20  20 103 162]]
             precision    recall  f1-score   support

          0       0.54      0.38      0.45       240
          1       0.35      0.42      0.38       286
          2       0.28      0.27      0.27       319
          3       0.49      0.53      0.51       305

avg / total       0.41      0.40      0.40      1150

---testing ends---
Time:  0.6845917804166675
-----epoch:  66 -----
--training begins--
--training ends--
--testing begins--
Epoch: 66 |train_ser_loss: 1.1150 |train_accuracy: 0.4738 |test_ser_loss: 1.2606 |test_accuracy: 0.4009
test: 
 [[ 93  81  40  26]
 [ 26 116  83  61]
 [ 32 115  84  88]
 [ 19  19  99 168]]
             precision    recall  f1-score   support

          0       0.55      0.39      0.45       240
          1       0.35      0.41 

--training ends--
--testing begins--
Epoch: 78 |train_ser_loss: 1.1075 |train_accuracy: 0.4795 |test_ser_loss: 1.2551 |test_accuracy: 0.4104
test: 
 [[101  74  46  19]
 [ 29 112  84  61]
 [ 31 112  90  86]
 [ 17  18 101 169]]
             precision    recall  f1-score   support

          0       0.57      0.42      0.48       240
          1       0.35      0.39      0.37       286
          2       0.28      0.28      0.28       319
          3       0.50      0.55      0.53       305

avg / total       0.42      0.41      0.41      1150

---testing ends---
Time:  0.7424321649596095
-----epoch:  79 -----
--training begins--
--training ends--
--testing begins--
Epoch: 79 |train_ser_loss: 1.0792 |train_accuracy: 0.4798 |test_ser_loss: 1.2537 |test_accuracy: 0.4130
test: 
 [[ 94  75  50  21]
 [ 28 112  79  67]
 [ 31 112  86  90]
 [ 16  18  88 183]]
             precision    recall  f1-score   support

          0       0.56      0.39      0.46       240
          1       0.35      0.39 

--training ends--
--testing begins--
Epoch: 91 |train_ser_loss: 1.0879 |train_accuracy: 0.4860 |test_ser_loss: 1.2559 |test_accuracy: 0.4191
test: 
 [[ 93  73  51  23]
 [ 22 110  90  64]
 [ 29 102 100  88]
 [ 13  16  97 179]]
             precision    recall  f1-score   support

          0       0.59      0.39      0.47       240
          1       0.37      0.38      0.37       286
          2       0.30      0.31      0.30       319
          3       0.51      0.59      0.54       305

avg / total       0.43      0.42      0.42      1150

---testing ends---
Time:  0.7242532940581441
-----epoch:  92 -----
--training begins--
--training ends--
--testing begins--
Epoch: 92 |train_ser_loss: 1.0558 |train_accuracy: 0.4894 |test_ser_loss: 1.2546 |test_accuracy: 0.4157
test: 
 [[ 94  73  51  22]
 [ 24 107  93  62]
 [ 29 103 100  87]
 [ 13  16  99 177]]
             precision    recall  f1-score   support

          0       0.59      0.39      0.47       240
          1       0.36      0.37 

In [ ]:
session 3

--layer 0--
Epoch: 99 |train_ser_loss: 0.7102 |train_accuracy: 0.6515 |test_ser_loss: 1.0781 |test_accuracy: 0.5391
test: 
 [[132  58  39  11]
 [ 29  83  91  83]
 [  9  62 154  94]
 [  1   3  50 251]]
             precision    recall  f1-score   support

          0       0.77      0.55      0.64       240
          1       0.40      0.29      0.34       286
          2       0.46      0.48      0.47       319
          3       0.57      0.82      0.67       305

avg / total       0.54      0.54      0.53      1150


--layer 1--
Epoch: 98 |train_ser_loss: 0.7233 |train_accuracy: 0.6775 |test_ser_loss: 1.0509 |test_accuracy: 0.5783
test: 
 [[144  46  38  12]
 [ 26  95 105  60]
 [ 17  49 191  62]
 [  3   6  61 235]]
             precision    recall  f1-score   support

          0       0.76      0.60      0.67       240
          1       0.48      0.33      0.39       286
          2       0.48      0.60      0.54       319
          3       0.64      0.77      0.70       305

avg / total       0.58      0.58      0.57      1150

--layer 2--
Epoch: 95 |train_ser_loss: 0.6407 |train_accuracy: 0.7156 |test_ser_loss: 1.0214 |test_accuracy: 0.6035
test: 
 [[156  33  35  16]
 [ 28  93 102  63]
 [ 21  23 224  51]
 [  5   5  74 221]]
             precision    recall  f1-score   support

          0       0.74      0.65      0.69       240
          1       0.60      0.33      0.42       286
          2       0.51      0.70      0.59       319
          3       0.63      0.72      0.67       305

avg / total       0.62      0.60      0.59      1150



--layer 3--
Epoch: 99 |train_ser_loss: 0.5950 |train_accuracy: 0.7301 |test_ser_loss: 1.0324 |test_accuracy: 0.6052
test: 
 [[158  32  36  14]
 [ 29 101  82  74]
 [ 26  35 203  55]
 [  4   5  62 234]]
             precision    recall  f1-score   support

          0       0.73      0.66      0.69       240
          1       0.58      0.35      0.44       286
          2       0.53      0.64      0.58       319
          3       0.62      0.77      0.69       305

avg / total       0.61      0.61      0.60      1150

--layer 4--
Epoch: 92 |train_ser_loss: 0.5702 |train_accuracy: 0.7499 |test_ser_loss: 0.9714 |test_accuracy: 0.6226
test: 
 [[162  27  39  12]
 [ 31 104  72  79]
 [ 23  19 212  65]
 [  5   3  59 238]]
             precision    recall  f1-score   support

          0       0.73      0.68      0.70       240
          1       0.68      0.36      0.47       286
          2       0.55      0.66      0.60       319
          3       0.60      0.78      0.68       305

avg / total       0.64      0.62      0.61      1150


--layer 5--
Epoch: 65 |train_ser_loss: 0.6071 |train_accuracy: 0.7400 |test_ser_loss: 0.9479 |test_accuracy: 0.6383
test: 
 [[171  25  37   7]
 [ 31 124  68  63]
 [ 34  28 199  58]
 [  3   8  54 240]]
             precision    recall  f1-score   support

          0       0.72      0.71      0.71       240
          1       0.67      0.43      0.53       286
          2       0.56      0.62      0.59       319
          3       0.65      0.79      0.71       305

avg / total       0.64      0.64      0.63      1150


--layer 6--
Epoch: 92 |train_ser_loss: 0.5583 |train_accuracy: 0.7674 |test_ser_loss: 0.9587 |test_accuracy: 0.6470
test: 
 [[170  26  36   8]
 [ 31 118  69  68]
 [ 30  28 211  50]
 [  6   7  47 245]]
             precision    recall  f1-score   support

          0       0.72      0.71      0.71       240
          1       0.66      0.41      0.51       286
          2       0.58      0.66      0.62       319
          3       0.66      0.80      0.72       305

avg / total       0.65      0.65      0.64      1150

--layer 7--
Epoch: 93 |train_ser_loss: 0.5306 |train_accuracy: 0.7738 |test_ser_loss: 0.9500 |test_accuracy: 0.6452
test: 
 [[161  30  37  12]
 [ 18 129  68  71]
 [ 27  34 202  56]
 [  3   8  44 250]]
             precision    recall  f1-score   support

          0       0.77      0.67      0.72       240
          1       0.64      0.45      0.53       286
          2       0.58      0.63      0.60       319
          3       0.64      0.82      0.72       305

avg / total       0.65      0.65      0.64      1150


--layer 8--
Epoch: 99 |train_ser_loss: 0.5837 |train_accuracy: 0.7598 |test_ser_loss: 0.9822 |test_accuracy: 0.6330
test: 
 [[156  27  43  14]
 [ 17 115  81  73]
 [ 23  28 210  58]
 [  5   7  46 247]]
             precision    recall  f1-score   support

          0       0.78      0.65      0.71       240
          1       0.65      0.40      0.50       286
          2       0.55      0.66      0.60       319
          3       0.63      0.81      0.71       305

avg / total       0.64      0.63      0.63      1150


--layer 9--
Epoch: 98 |train_ser_loss: 0.6380 |train_accuracy: 0.7366 |test_ser_loss: 0.9978 |test_accuracy: 0.6183
test: 
 [[151  26  52  11]
 [ 17 119  80  70]
 [ 27  27 209  56]
 [  6   8  59 232]]
             precision    recall  f1-score   support

          0       0.75      0.63      0.68       240
          1       0.66      0.42      0.51       286
          2       0.52      0.66      0.58       319
          3       0.63      0.76      0.69       305

avg / total       0.63      0.62      0.61      1150


--layer 10--
Epoch: 99 |train_ser_loss: 0.6975 |train_accuracy: 0.6570 |test_ser_loss: 1.0741 |test_accuracy: 0.5643
test: 
 [[122  44  62  12]
 [ 14 102  95  75]
 [  8  40 182  89]
 [ 11   6  45 243]]
             precision    recall  f1-score   support

          0       0.79      0.51      0.62       240
          1       0.53      0.36      0.43       286
          2       0.47      0.57      0.52       319
          3       0.58      0.80      0.67       305

avg / total       0.58      0.56      0.56      1150

--layer 11--
Epoch: 85 |train_ser_loss: 1.0383 |train_accuracy: 0.4961 |test_ser_loss: 1.2377 |test_accuracy: 0.4357
test: 
 [[101  63  64  12]
 [ 30  85 103  68]
 [ 29  84 113  93]
 [ 20  17  66 202]]
             precision    recall  f1-score   support

          0       0.56      0.42      0.48       240
          1       0.34      0.30      0.32       286
          2       0.33      0.35      0.34       319
          3       0.54      0.66      0.59       305

avg / total       0.44      0.44      0.43      1150

--layer 12--
Epoch: 91 |train_ser_loss: 1.0879 |train_accuracy: 0.4860 |test_ser_loss: 1.2559 |test_accuracy: 0.4191
test: 
 [[ 93  73  51  23]
 [ 22 110  90  64]
 [ 29 102 100  88]
 [ 13  16  97 179]]
             precision    recall  f1-score   support

          0       0.59      0.39      0.47       240
          1       0.37      0.38      0.37       286
          2       0.30      0.31      0.30       319
          3       0.51      0.59      0.54       305

avg / total       0.43      0.42      0.42      1150

In [ ]:
session 1

--layer 0--
Epoch: 97 |train_ser_loss: 0.3623 |train_accuracy: 0.6460 |test_ser_loss: 1.0066 |test_accuracy: 0.5503
test: 
 [[148  44  26  11]
 [ 34  99 111  26]
 [  5  44 176 157]
 [  4   2  19 168]]
             precision    recall  f1-score   support

          0       0.77      0.65      0.70       229
          1       0.52      0.37      0.43       270
          2       0.53      0.46      0.49       382
          3       0.46      0.87      0.61       193

avg / total       0.57      0.55      0.54      1074


--layer 1--
Epoch: 99 |train_ser_loss: 0.1650 |train_accuracy: 0.6814 |test_ser_loss: 0.9590 |test_accuracy: 0.6043
test: 
 [[150  43  23  13]
 [ 20 151  70  29]
 [ 10  48 179 145]
 [  4   2  18 169]]
             precision    recall  f1-score   support

          0       0.82      0.66      0.73       229
          1       0.62      0.56      0.59       270
          2       0.62      0.47      0.53       382
          3       0.47      0.88      0.62       193

avg / total       0.63      0.60      0.60      1074

--layer 2--
Epoch: 97 |train_ser_loss: 0.0750 |train_accuracy: 0.7110 |test_ser_loss: 0.9655 |test_accuracy: 0.6229
test: 
 [[152  37  31   9]
 [ 14 152  72  32]
 [ 11  36 193 142]
 [  5   1  15 172]]
             precision    recall  f1-score   support

          0       0.84      0.66      0.74       229
          1       0.67      0.56      0.61       270
          2       0.62      0.51      0.56       382
          3       0.48      0.89      0.63       193

avg / total       0.65      0.62      0.62      1074

--layer 3--
Epoch: 42 |train_ser_loss: 0.1702 |train_accuracy: 0.6943 |test_ser_loss: 0.9268 |test_accuracy: 0.6266
test: 
 [[152  32  30  15]
 [  9 162  59  40]
 [ 12  30 180 160]
 [  5   3   6 179]]
             precision    recall  f1-score   support

          0       0.85      0.66      0.75       229
          1       0.71      0.60      0.65       270
          2       0.65      0.47      0.55       382
          3       0.45      0.93      0.61       193

avg / total       0.68      0.63      0.63      1074

--layer 4--
Epoch: 93 |train_ser_loss: 0.1202 |train_accuracy: 0.7085 |test_ser_loss: 0.9271 |test_accuracy: 0.6378
test: 
 [[146  44  27  12]
 [ 11 155  76  28]
 [  7  39 214 122]
 [  2   2  19 170]]
             precision    recall  f1-score   support

          0       0.88      0.64      0.74       229
          1       0.65      0.57      0.61       270
          2       0.64      0.56      0.60       382
          3       0.51      0.88      0.65       193

avg / total       0.67      0.64      0.64      1074


--layer 5--
Epoch: 90 |train_ser_loss: 0.0712 |train_accuracy: 0.7404 |test_ser_loss: 0.8727 |test_accuracy: 0.6601
test: 
 [[139  46  35   9]
 [  6 171  63  30]
 [  7  28 221 126]
 [  1   4  10 178]]
             precision    recall  f1-score   support

          0       0.91      0.61      0.73       229
          1       0.69      0.63      0.66       270
          2       0.67      0.58      0.62       382
          3       0.52      0.92      0.66       193

avg / total       0.70      0.66      0.66      1074


--layer 6--
Epoch: 63 |train_ser_loss: 0.0795 |train_accuracy: 0.7427 |test_ser_loss: 0.8658 |test_accuracy: 0.6667
test: 
 [[156  26  37  10]
 [  7 182  51  30]
 [  8  35 205 134]
 [  2   4  14 173]]
             precision    recall  f1-score   support

          0       0.90      0.68      0.78       229
          1       0.74      0.67      0.70       270
          2       0.67      0.54      0.60       382
          3       0.50      0.90      0.64       193

avg / total       0.70      0.67      0.67      1074


--layer 7--
Epoch: 99 |train_ser_loss: 0.0466 |train_accuracy: 0.7501 |test_ser_loss: 0.9238 |test_accuracy: 0.6601
test: 
 [[154  31  36   8]
 [  5 175  62  28]
 [ 14  19 211 138]
 [  1   2  21 169]]
             precision    recall  f1-score   support

          0       0.89      0.67      0.76       229
          1       0.77      0.65      0.70       270
          2       0.64      0.55      0.59       382
          3       0.49      0.88      0.63       193

avg / total       0.70      0.66      0.66      1074

--layer 8--
Epoch: 78 |train_ser_loss: 0.1067 |train_accuracy: 0.7528 |test_ser_loss: 0.8848 |test_accuracy: 0.6657
test: 
 [[164  27  31   7]
 [  8 187  43  32]
 [ 14  41 196 131]
 [  2   7  16 168]]
             precision    recall  f1-score   support

          0       0.87      0.72      0.79       229
          1       0.71      0.69      0.70       270
          2       0.69      0.51      0.59       382
          3       0.50      0.87      0.63       193

avg / total       0.70      0.67      0.67      1074

--layer 9--
Epoch: 97 |train_ser_loss: 0.1680 |train_accuracy: 0.7248 |test_ser_loss: 0.8691 |test_accuracy: 0.6499
test: 
 [[165  25  29  10]
 [  8 165  63  34]
 [ 15  31 204 132]
 [  4   3  22 164]]
             precision    recall  f1-score   support

          0       0.86      0.72      0.78       229
          1       0.74      0.61      0.67       270
          2       0.64      0.53      0.58       382
          3       0.48      0.85      0.62       193

avg / total       0.68      0.65      0.65      1074

--layer 10--
Epoch: 96 |train_ser_loss: 0.2384 |train_accuracy: 0.6229 |test_ser_loss: 0.9488 |test_accuracy: 0.5857
test: 
 [[159  31  31   8]
 [ 19 134  84  33]
 [ 11  51 170 150]
 [  3   0  24 166]]
             precision    recall  f1-score   support

          0       0.83      0.69      0.76       229
          1       0.62      0.50      0.55       270
          2       0.55      0.45      0.49       382
          3       0.46      0.86      0.60       193

avg / total       0.61      0.59      0.58      1074

--layer 11--
Epoch: 98 |train_ser_loss: 0.5452 |train_accuracy: 0.5079 |test_ser_loss: 1.0761 |test_accuracy: 0.5279
test: 
 [[146  30  36  17]
 [ 26 162  45  37]
 [ 36  77 101 168]
 [  4   8  23 158]]
             precision    recall  f1-score   support

          0       0.69      0.64      0.66       229
          1       0.58      0.60      0.59       270
          2       0.49      0.26      0.34       382
          3       0.42      0.82      0.55       193

avg / total       0.54      0.53      0.51      1074

--layer 12--
Epoch: 81 |train_ser_loss: 0.7205 |train_accuracy: 0.4577 |test_ser_loss: 1.1474 |test_accuracy: 0.4655
test: 
 [[118  38  53  20]
 [ 24 168  28  50]
 [ 40 100  59 183]
 [  5  19  14 155]]
             precision    recall  f1-score   support

          0       0.63      0.52      0.57       229
          1       0.52      0.62      0.56       270
          2       0.38      0.15      0.22       382
          3       0.38      0.80      0.52       193

avg / total       0.47      0.47      0.43      1074

In [ ]:
session 2

--layer 0--
Epoch: 59 |train_ser_loss: 0.8226 |train_accuracy: 0.6088 |test_ser_loss: 0.9236 |test_accuracy: 0.6043
test: 
 [[ 88  37  11   1]
 [ 25 179 105  16]
 [ 11  76 242  33]
 [  6   3  80 108]]
             precision    recall  f1-score   support

          0       0.68      0.64      0.66       137
          1       0.61      0.55      0.58       325
          2       0.55      0.67      0.60       362
          3       0.68      0.55      0.61       197

avg / total       0.61      0.60      0.60      1021


--layer 1--
Epoch: 99 |train_ser_loss: 0.6695 |train_accuracy: 0.7028 |test_ser_loss: 0.8498 |test_accuracy: 0.6494
test: 
 [[ 96  25  16   0]
 [ 25 219  71  10]
 [ 10  98 227  27]
 [  3  13  60 121]]
             precision    recall  f1-score   support

          0       0.72      0.70      0.71       137
          1       0.62      0.67      0.64       325
          2       0.61      0.63      0.62       362
          3       0.77      0.61      0.68       197

avg / total       0.66      0.65      0.65      1021

--layer 2--
Epoch: 95 |train_ser_loss: 0.5665 |train_accuracy: 0.7240 |test_ser_loss: 0.8567 |test_accuracy: 0.6327
test: 
 [[103  22  12   0]
 [ 32 200  82  11]
 [ 16  99 225  22]
 [  5  15  59 118]]
             precision    recall  f1-score   support

          0       0.66      0.75      0.70       137
          1       0.60      0.62      0.61       325
          2       0.60      0.62      0.61       362
          3       0.78      0.60      0.68       197

avg / total       0.64      0.63      0.63      1021


--layer 3--
Epoch: 78 |train_ser_loss: 0.6254 |train_accuracy: 0.7285 |test_ser_loss: 0.8485 |test_accuracy: 0.6445
test: 
 [[ 87  32  18   0]
 [ 17 207  87  14]
 [  7  91 242  22]
 [  1  17  57 122]]
             precision    recall  f1-score   support

          0       0.78      0.64      0.70       137
          1       0.60      0.64      0.62       325
          2       0.60      0.67      0.63       362
          3       0.77      0.62      0.69       197

avg / total       0.66      0.64      0.65      1021

--layer 4--
Epoch: 98 |train_ser_loss: 0.5779 |train_accuracy: 0.7417 |test_ser_loss: 0.8653 |test_accuracy: 0.6621
test: 
 [[ 93  33  10   1]
 [ 17 239  59  10]
 [  7 105 225  25]
 [  2  15  61 119]]
             precision    recall  f1-score   support

          0       0.78      0.68      0.73       137
          1       0.61      0.74      0.67       325
          2       0.63      0.62      0.63       362
          3       0.77      0.60      0.68       197

avg / total       0.67      0.66      0.66      1021

--layer 5--
Epoch: 98 |train_ser_loss: 0.6393 |train_accuracy: 0.7205 |test_ser_loss: 0.9000 |test_accuracy: 0.6327
test: 
 [[103  26   8   0]
 [ 22 208  90   5]
 [  9 101 231  21]
 [  1   9  83 104]]
             precision    recall  f1-score   support

          0       0.76      0.75      0.76       137
          1       0.60      0.64      0.62       325
          2       0.56      0.64      0.60       362
          3       0.80      0.53      0.64       197

avg / total       0.65      0.63      0.63      1021

--layer 6--
Epoch: 84 |train_ser_loss: 0.6556 |train_accuracy: 0.7098 |test_ser_loss: 0.8398 |test_accuracy: 0.6592
test: 
 [[111  21   4   1]
 [ 26 224  66   9]
 [ 11 102 228  21]
 [  3   3  81 110]]
             precision    recall  f1-score   support

          0       0.74      0.81      0.77       137
          1       0.64      0.69      0.66       325
          2       0.60      0.63      0.62       362
          3       0.78      0.56      0.65       197

avg / total       0.67      0.66      0.66      1021

--layer 7--
Epoch: 65 |train_ser_loss: 0.6386 |train_accuracy: 0.7305 |test_ser_loss: 0.8309 |test_accuracy: 0.6709
test: 
 [[110  18   8   1]
 [ 27 233  48  17]
 [ 21 102 214  25]
 [  3  22  44 128]]
             precision    recall  f1-score   support

          0       0.68      0.80      0.74       137
          1       0.62      0.72      0.67       325
          2       0.68      0.59      0.63       362
          3       0.75      0.65      0.70       197

avg / total       0.68      0.67      0.67      1021

--layer 8--
Epoch: 99 |train_ser_loss: 0.6520 |train_accuracy: 0.7484 |test_ser_loss: 0.9087 |test_accuracy: 0.6366
test: 
 [[110  15  11   1]
 [ 25 230  60  10]
 [ 21 112 201  28]
 [  4  21  63 109]]
             precision    recall  f1-score   support

          0       0.69      0.80      0.74       137
          1       0.61      0.71      0.65       325
          2       0.60      0.56      0.58       362
          3       0.74      0.55      0.63       197

avg / total       0.64      0.64      0.63      1021

--layer 9--
Epoch: 77 |train_ser_loss: 0.6811 |train_accuracy: 0.7127 |test_ser_loss: 0.8886 |test_accuracy: 0.6259
test: 
 [[105  23   9   0]
 [ 20 244  47  14]
 [ 32 129 181  20]
 [  5  36  47 109]]
             precision    recall  f1-score   support

          0       0.65      0.77      0.70       137
          1       0.56      0.75      0.64       325
          2       0.64      0.50      0.56       362
          3       0.76      0.55      0.64       197

avg / total       0.64      0.63      0.62      1021

--layer 10--
Epoch: 98 |train_ser_loss: 0.8573 |train_accuracy: 0.6446 |test_ser_loss: 0.9683 |test_accuracy: 0.5661
test: 
 [[105  24   6   2]
 [ 33 183 100   9]
 [ 25 133 189  15]
 [ 10   4  82 101]]
             precision    recall  f1-score   support

          0       0.61      0.77      0.68       137
          1       0.53      0.56      0.55       325
          2       0.50      0.52      0.51       362
          3       0.80      0.51      0.62       197

avg / total       0.58      0.57      0.57      1021

--layer 11--
Epoch: 88 |train_ser_loss: 1.1568 |train_accuracy: 0.4816 |test_ser_loss: 1.1461 |test_accuracy: 0.4417
test: 
 [[101  29   6   1]
 [ 66 189  63   7]
 [104 155  76  27]
 [ 29  21  62  85]]
             precision    recall  f1-score   support

          0       0.34      0.74      0.46       137
          1       0.48      0.58      0.53       325
          2       0.37      0.21      0.27       362
          3       0.71      0.43      0.54       197

avg / total       0.46      0.44      0.43      1021

--layer 12--
Epoch: 99 |train_ser_loss: 1.1301 |train_accuracy: 0.4914 |test_ser_loss: 1.1533 |test_accuracy: 0.4319
test: 
 [[ 83  43   4   7]
 [ 52 221  26  26]
 [ 81 192  43  46]
 [ 28  29  46  94]]
             precision    recall  f1-score   support

          0       0.34      0.61      0.44       137
          1       0.46      0.68      0.55       325
          2       0.36      0.12      0.18       362
          3       0.54      0.48      0.51       197

avg / total       0.42      0.43      0.39      1021